In [194]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from signals import *
from frequencyestimator import *
from scipy.optimize import basinhopping, minimize
from tqdm.auto import tqdm

sns.set_style("whitegrid")
sns.despine(left=True, bottom=True)
sns.set_context("poster", font_scale = .45, rc={"grid.linewidth": 0.8})

<Figure size 640x480 with 0 Axes>

In [16]:
P0 = lambda n, theta: np.cos((2*n+1)*theta)**2
P1 = lambda n, theta: np.sin((2*n+1)*theta)**2

def estimate_signal(depths, n_samples, theta, eta=0.0):
        signals = np.zeros(len(depths), dtype = np.complex128)
        measurements = np.zeros(len(depths))
        for i,n in enumerate(depths):
            # Get the exact measuremnt probabilities
            p0 = P0(n, theta)
            p1 = P1(n, theta)

            p0x = P0x(n,theta)
            p1x = P1x(n,theta)

            # Get the "noisy" probabilities by sampling and adding a bias term that pushes towards 50/50 mixture
            eta_n = (1.0-eta)**(n+1) # The error at depth n increases as more queries are implemented
            p0_estimate = np.random.binomial(n_samples[i], eta_n*p0 + (1.0-eta_n)*0.5)/n_samples[i]
            p1_estimate = 1 - p0_estimate

            p0x_estimate = np.random.binomial(n_samples[i], eta_n*p0x + (1.0-eta_n)*0.5)/n_samples[i]
            p1x_estimate = 1.0 - p0x_estimate
            measurements[i] = p0_estimate
            
            # Estimate theta
            theta_estimated = np.arctan2(p0x_estimate - p1x_estimate, p0_estimate - p1_estimate)

            # Compute f(n) - Eq. 3
            fi_estimate = np.exp(1.0j*theta_estimated)
            signals[i] = fi_estimate
         
        return signals, measurements

from scipy.stats import binom

def apply_correction(ula_signal, measurements, theta_est, theta):
    p_exact = np.cos((2*ula_signal.depths+1)*(theta))**2
    p_neg = np.cos((2*ula_signal.depths+1)*(-theta))**2
    p_o2 = np.cos((2 * ula_signal.depths + 1) * (theta_est/2.0)) ** 2
    p_o4 = np.cos((2 * ula_signal.depths + 1) * (theta_est/4.0)) ** 2
    p_same = np.cos((2*ula_signal.depths+1)*(theta_est))**2
    p_s2 = np.cos((2 * ula_signal.depths + 1) * (np.pi/2-theta_est)) ** 2
    p_s4 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 4 - theta_est)) ** 2
    p_s2_o2 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 2 - theta_est/2)) ** 2
    p_s4_o2 = np.cos((2 * ula_signal.depths + 1) * (np.pi / 4 - theta_est/2)) ** 2

    l_exact = np.sum(
        np.log([1e-75 + binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk],
                                  p_exact[kk]) for kk in
                range(len(ula_signal.n_samples))]))
    l_neg = np.sum(
        np.log([1e-75 + binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk],
                                  p_neg[kk]) for kk in
                range(len(ula_signal.n_samples))]))
    l_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk]*measurements[kk], ula_signal.n_samples[kk], p_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_o4 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_o4[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_same = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_same[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s4 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s4[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s2_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s2_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    l_s4_o2 = np.sum(
        np.log([1e-75+binom.pmf(ula_signal.n_samples[kk] * measurements[kk], ula_signal.n_samples[kk], p_s4_o2[kk]) for kk in
         range(len(ula_signal.n_samples))]))
    
    which_correction = np.argmax([l_same, l_s2, l_s4, l_o2, l_o4, l_s2_o2, l_s4_o2, l_neg])
    if which_correction == 1:
        theta_est = np.pi/2.0 - theta_est
    elif which_correction == 2:
        theta_est = np.pi/4.0 - theta_est
    elif which_correction == 3:
        theta_est = theta_est/2
    elif which_correction == 4:
        theta_est = theta_est/4
    elif which_correction == 5:
        theta_est = np.pi / 2.0 - 0.5 * theta_est
    elif which_correction == 6:
        theta_est = np.pi / 4.0 - 0.5 * theta_est
    # elif which_correction == 7:
    #     theta_est = -theta_est

    # print(f'FINAL ANGLE FOUND: {theta_est, theta}')

    return theta_est

## Use Basinhopping (simulated annealing) from scipy.optimize to minimize the objective function

In [3]:
def objective_function(lp, cos_signal, abs_sin, ula_signal, esprit):   
    signal = cos_signal + 1.0j * lp * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    _, _ = esprit.estimate_theta_toeplitz(R)
    eigs = np.abs(esprit.eigs)[:2]
    obj = eigs[1] - eigs[0]

    return obj

## Function to implement the basinhopping based csAE

In [9]:
a = 0.2
theta = np.arcsin(a)

narray = [2,2,2,2,2,2,2,3]
ula_signal = TwoqULASignal(M=narray, C=3)
esprit = ESPIRIT()
# narray = [2]*8

def csae_with_basinhopping(theta, ula_signal, esprit, niter, T=100.0, disp=False):

    depths = ula_signal.depths
    n_samples = ula_signal.n_samples
    csignal, measurements = estimate_signal(depths, n_samples, theta)

    cos_signal = np.real(csignal)

    correct_signs = np.sign(np.imag(csignal))
    abs_sin = np.abs(np.imag(csignal))

    # init_signs = np.array([1.0 for _ in range(len(correct_signs))])
    init_signs = (-1)**np.random.binomial(1, 0.5, len(correct_signs))
    bounds=[[-1,1] for _ in range(len(init_signs))]
    res = basinhopping(objective_function, init_signs, niter=niter, T=T, stepsize=1.0, minimizer_kwargs={'args': (cos_signal, abs_sin, ula_signal, esprit), 'bounds': bounds, 'method': 'COBYLA', 'options': {'maxiter': 20}}, disp=disp)

    if disp:
        print(res)

    x = res.x
    signal = cos_signal + 1.0j * x * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    theta_est, _ = esprit.estimate_theta_toeplitz(R)
    theta_est = apply_correction(ula_signal, measurements, theta_est, theta) #apply correction
    eigs = np.abs(esprit.eigs)[:2]
    basin_obj = eigs[1] - eigs[0]
    if disp:
        print(f'basin obj: {basin_obj}')

    cR = ula_signal.get_cov_matrix_toeplitz(csignal)
    theta_est1, _ = esprit.estimate_theta_toeplitz(cR)
    eigs = np.abs(esprit.eigs)[:2]
    true_obj = eigs[1] - eigs[0]
    if disp:
        print(f'true obj: {true_obj}')

    num_queries = np.sum(np.array(ula_signal.depths)*np.array(ula_signal.n_samples)) + ula_signal.n_samples[0]
    # Compute the maximum single query
    max_single_query = np.max(ula_signal.depths)

    ret_dict = {'theta_est': theta_est, 'theta_est1': theta_est1, 
                'error': np.abs(np.sin(theta)-np.sin(theta_est)), 
                'error1': np.abs(np.sin(theta)-np.sin(theta_est1)), 
                'queries': num_queries, 'depth': max_single_query,
                'true_obj': true_obj, 'basin_obj': basin_obj}
    
    return ret_dict

csae_with_basinhopping(theta=theta, ula_signal=ula_signal, esprit=esprit, niter=30, T=100.0, disp=True)

basinhopping step 0: f -93.0809
current x: [-1.          1.         -0.33640318  1.         -1.          0.25190959
  1.         -1.         -0.5        -1.        ]
basinhopping step 1: f -189.563 trial_f -189.563 accepted 1  lowest_f -93.0809
current x: [-1.          1.         -0.33640318  1.         -1.          0.25190959
  1.         -1.         -0.5        -1.        ]
basinhopping step 2: f -189.563 trial_f -127.364 accepted 0  lowest_f -93.0809
current x: [-1.          1.         -0.33640318  1.         -1.          0.25190959
  1.         -1.         -0.5        -1.        ]
basinhopping step 3: f -356.597 trial_f -356.597 accepted 1  lowest_f -93.0809
current x: [-1.          1.         -0.33640318  1.         -1.          0.25190959
  1.         -1.         -0.5        -1.        ]
basinhopping step 4: f -284.223 trial_f -284.223 accepted 1  lowest_f -93.0809
current x: [-1.          1.         -0.33640318  1.         -1.          0.25190959
  1.         -1.         -0.5   

{'theta_est': 0.6116051242915076,
 'theta_est1': 0.1998667539860781,
 'error': 0.37418235850489584,
 'error1': 0.001461260933065145,
 'queries': 3069,
 'depth': 256,
 'true_obj': -445.89955401668954,
 'basin_obj': -93.08094242990023}

## Let's do some statistics now

In [13]:
avals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
avals = [0.9]
# avals = [0.5, 0.6, 0.7, 0.8, 0.9]
narray = [2,2,2,2,2,3]

num_queries = np.zeros(len(avals), dtype=int)
max_single_query = np.zeros(len(avals), dtype=int)

num_mc=100
thetas = np.zeros((len(avals), num_mc))
errors = np.zeros((len(avals), num_mc))
thetas1 = np.zeros((len(avals), num_mc))
errors1 = np.zeros((len(avals), num_mc))

basin_obj = np.zeros((len(avals), num_mc))
true_obj = np.zeros((len(avals), num_mc))

ula_signal = TwoqULASignal(M=narray, C=3)
esprit = ESPIRIT()

for j,a in enumerate(avals):
    theta = np.arcsin(a)
    print(f'theta = {theta}')
    for i in tqdm(range(num_mc)):
        res = csae_with_basinhopping(theta=theta,ula_signal=ula_signal, esprit=esprit, niter=30, T=100.0, disp=False)
        thetas[j][i] = res['theta_est']
        errors[j][i] = res['error']

        thetas1[j][i] = res['theta_est1']
        errors1[j][i] = res['error1']

        basin_obj[j][i] = res['basin_obj']
        true_obj[j][i] = res['true_obj']
    num_queries[j] = res['queries']
    max_single_query[j] = res['depth']

    print(f'constant factors query and depth: {np.percentile(errors[j], 68) * num_queries[j]}, {np.percentile(errors[j], 68) * max_single_query[j]}')




theta = 1.1197695149986342


100%|██████████| 100/100 [02:35<00:00,  1.55s/it]

constant factors query and depth: 193.18480395217995, 16.161865951554923


In [ ]:
errors68 = np.percentile(errors, 68, axis=1)
print(f'constant factors for query complexity: {errors68*num_queries}'),
print(f'constant factors for max depth complexity: {errors68*max_single_query}')

constant factors for query complexity: [294.69381801]
constant factors for max depth complexity: [24.65412334]


In [ ]:
thetas

array([[ 0.96229847,  0.18864297,  0.1755189 ,  1.3826461 ,  0.17707869,
         0.21409398,  1.40640757,  0.20301506,  1.36839073,  0.59532869,
         0.2037833 ,  0.9845041 ,  0.17986687,  0.17735446,  0.18981146,
         0.60584376,  0.21389457, -0.19738589,  1.38406963,  0.69386201,
         0.1548344 ,  1.40773211,  1.39304026,  0.61851163,  0.6225561 ,
         0.17980919,  0.20054314,  0.6076405 ,  0.20156756,  0.20237674,
         0.20231301,  1.35757162,  0.59709053,  0.18984185,  0.19766042,
        -0.19943748,  0.18648793,  0.62519207,  0.2175485 ,  0.17939451,
         0.22845861,  0.16766145,  1.40875046,  0.1859709 ,  0.20455261,
        -0.19955915,  0.97515715,  0.2020814 ,  0.186517  ,  0.20226509,
         0.6073727 ,  0.98343894,  0.97836063,  0.19431052,  0.5939019 ,
        -0.16533705,  0.15536816,  0.18768143,  0.20016689,  0.19909627,
         0.18551123,  1.38187677,  0.18514402,  0.96058869,  0.9786671 ,
         0.97245861,  0.18044569,  0.20527369, -0.2

In [ ]:
np.arcsin(avals)

array([0.20135792])

In [14]:
np.sum(np.where(np.array(basin_obj[0])/np.array(true_obj[0]) >= 0.99, 1, 0))/len(basin_obj[0])

0.07

In [15]:
np.array(basin_obj)/np.array(true_obj)

array([[0.72046427, 0.35984478, 0.73409009, 0.76176494, 0.69820147,
        0.54635501, 0.35935237, 0.34021621, 0.20363589, 0.75440002,
        0.37233736, 0.2636801 , 0.62406487, 0.33958581, 0.30520346,
        0.23635487, 0.45178856, 0.26139783, 0.28924015, 0.39073059,
        0.37502253, 0.30408165, 1.        , 0.57190987, 0.77179649,
        0.59988808, 0.23311264, 0.73000865, 0.30693373, 0.24378804,
        0.51457092, 0.22869077, 0.91763879, 0.47426253, 0.31687221,
        0.35496062, 0.25372006, 0.22306779, 0.53632604, 0.62462682,
        0.44701812, 0.91424244, 0.13266017, 0.59750083, 0.52924915,
        0.57127971, 0.33667738, 0.9084274 , 0.32838735, 0.25799766,
        1.        , 0.32777188, 0.89662065, 0.35688064, 0.40362583,
        0.21907848, 0.13655315, 0.32175364, 0.28485943, 0.0872619 ,
        0.43663995, 0.91190779, 0.7845106 , 0.63637356, 0.7101897 ,
        0.24975669, 0.57147796, 0.91005814, 0.19156878, 0.59926923,
        0.47383754, 1.07503291, 0.46042226, 1.01

## What is the distribution of the signs? 

In [228]:
narray = [2,2,2,2,2,2,2,3]

# avals = np.linspace(0.1, 0.9, 8)
avals = [0.1*i for i in range(1, 10)]

ula_signal = TwoqULASignal(M=narray, C=5)
esprit = ESPIRIT()

num_mc = 1000

sign_distributions = {}

for a in avals:
    theta = np.arcsin(a)
    distr = {}
    for i in range(num_mc):
        signal, _ = estimate_signal(ula_signal.depths, ula_signal.n_samples, theta)
        # print(measurements)
        signs = tuple(np.sign(np.imag(signal)))
        # print(signs)

        distr[signs] = distr.get(signs, 0.0) + 1/num_mc
    sign_distributions[str(a)] = distr



In [229]:
2**len(ula_signal.depths), len(sign_distributions['0.1'])

(1024, 20)

In [230]:
all_signs = set()
for a in avals:
    distr = sign_distributions[str(a)]
    for sign in distr.keys():
        all_signs.add(sign)
len(all_signs)

124

In [231]:
avals

[0.1,
 0.2,
 0.30000000000000004,
 0.4,
 0.5,
 0.6000000000000001,
 0.7000000000000001,
 0.8,
 0.9]

In [232]:
len(all_signs)/2**len(ula_signal.depths)

0.12109375

In [376]:
narray = [2,2,2,2,2,3]

def get_heavy_signs(narray, steps):

    avals = np.linspace(0.1, 0.9, steps)

    ula_signal = TwoqULASignal(M=narray, C=5)

    num_mc = 1000

    sign_distributions = {}

    for a in avals:
        theta = np.arcsin(a)
        distr = {}
        for i in range(num_mc):
            signal, _ = estimate_signal(ula_signal.depths, ula_signal.n_samples, theta)
            # print(measurements)
            signs = tuple(np.sign(np.imag(signal)))
            # print(signs)

            distr[signs] = distr.get(signs, 0.0) + 1/num_mc
        sign_distributions[str(a)] = distr

    def get_signs(sign_distribution):
        # Sort the dictionary by values in descending order
        sorted_data = dict(sorted(sign_distribution.items(), key=lambda item: item[1], reverse=True))

        # Create a new dictionary with cumulative sum close to 0.68
        cumulative_dict = {}
        cumulative_sum = 0.0

        for key, value in sorted_data.items():
            cumulative_dict[key] = value
            if cumulative_sum + value > 0.68:
                break
            cumulative_sum += value

        # Output the result
        return cumulative_dict
    
    ret_dict = {}
    for a in avals:
        distr = get_signs(sign_distributions[str(a)])
        ret_dict[str(a)] = distr
 
    # Normalize the values
    normalized_data = {}
    for key, sub_dict in ret_dict.items():
        total_sum = sum(sub_dict.values())
        normalized_data[key] = {k: v / total_sum for k, v in sub_dict.items()}

    return normalized_data

normalized_data = get_heavy_signs(narray, 10)


In [388]:
import ast
# Taking a random sample based on the normalized probabilities
def sample_from_normalized(data, sample_size=1):
    keys = list(map(str, list(data.keys())))
    probabilities = list(data.values())
    sampled_keys = np.random.choice(a=keys, size=sample_size, p=probabilities)
    return [ast.literal_eval(t) for t in sampled_keys]
avals = list(normalized_data.keys())
# Get one random sample
sampled = sample_from_normalized(normalized_data[avals[0]], sample_size=3)
print("Sampled keys:",sampled)

Sampled keys: [(1.0, 1.0, 1.0, 1.0, -1.0, -1.0, 1.0, 1.0), (1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0), (1.0, 1.0, 1.0, 1.0, -1.0, 1.0, 1.0, 1.0)]


In [379]:
aa_milne_arr = ['pooh', 'rabbit', 'piglet', 'Christopher']
np.random.choice(aa_milne_arr, 5, p=[0.5, 0.1, 0.1, 0.3])

array(['Christopher', 'pooh', 'pooh', 'Christopher', 'piglet'],
      dtype='<U11')

In [418]:
np.random.seed(42)

a = np.random.uniform(0.1, 0.9)
theta = np.arcsin(a)

narray = [2,2,2,2,2,2,2,3]
heavy_signs = get_heavy_signs(narray, len(narray)**2)

ula_signal = TwoqULASignal(M=narray, C=3)
esprit = ESPIRIT()

def csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=False, correction=True, optimize=False, method='COBYLA', maxiter=20, disp=False):

    depths = ula_signal.depths
    n_samples = ula_signal.n_samples
    csignal, measurements = estimate_signal(depths, n_samples, theta)

    cos_signal = np.real(csignal)

    correct_signs = np.sign(np.imag(csignal))
    abs_sin = np.abs(np.imag(csignal))

    bounds=[[-1,1] for _ in range(len(depths))]

    avg_nfev = 0.0
    obj = 1.0
    if sample:
        avals = list(heavy_signs.keys())
        signs_to_try = []
        for a in avals:
            signs_to_try += sample_from_normalized(heavy_signs[a], sample_size=5)
        signs_to_try = list(set(signs_to_try))
        if disp:
            print(f'number of signs to try: {len(signs_to_try)}')

        for x in signs_to_try:
            if optimize:
                res = minimize(objective_function, x, args=(cos_signal, abs_sin, ula_signal, esprit), bounds=bounds, method=method, options={'maxiter': maxiter, 'disp': False})
                avg_nfev += res.nfev
                if disp:
                    print(res.x)
                if disp:
                    print(f'number fev: {res.nfev}')
                if res.fun < obj:
                    obj = res.fun
                    # x_star = np.sign(res.x)
                    x_star = res.x
            else:
                signal = cos_signal + 1.0j * np.array(x) * abs_sin
                R = ula_signal.get_cov_matrix_toeplitz(signal)
                _, _ = esprit.estimate_theta_toeplitz(R)
                eigs = np.abs(esprit.eigs)[:2]
                basin_obj = eigs[1] - eigs[0]
                if basin_obj < obj:
                    obj = basin_obj
                    x_star = np.array(x)
    else:
        signs_to_try = []
        for a in heavy_signs.keys():
            signs = heavy_signs[a].keys()
            signs_to_try += signs
        signs_to_try = list(set(signs_to_try))
        if disp:
            print(f'number of signs to try: {len(signs_to_try)}')
        for x in signs_to_try:
            if optimize:
                res = minimize(objective_function, x, args=(cos_signal, abs_sin, ula_signal, esprit), bounds=bounds, method=method, options={'maxiter': maxiter, 'disp': False})
                avg_nfev += res.nfev
                if disp:
                    print(res.x)
                if disp:
                    print(f'number fev: {res.nfev}')
                if res.fun < obj:
                    obj = res.fun
                    # x_star = np.sign(res.x)
                    x_star = np.sign(res.x)
            else:
                signal = cos_signal + 1.0j * np.array(x) * abs_sin
                R = ula_signal.get_cov_matrix_toeplitz(signal)
                _, _ = esprit.estimate_theta_toeplitz(R)
                eigs = np.abs(esprit.eigs)[:2]
                basin_obj = eigs[1] - eigs[0]
                if basin_obj < obj:
                    obj = basin_obj
                    x_star = np.array(x)
    if disp:
        print(x_star)
    signal = cos_signal + 1.0j * x_star * abs_sin
    R = ula_signal.get_cov_matrix_toeplitz(signal)
    theta_est, _ = esprit.estimate_theta_toeplitz(R)
    if correction:
        theta_est = apply_correction(ula_signal, measurements, theta_est, theta) #apply correction
    eigs = np.abs(esprit.eigs)[:2]
    basin_obj = eigs[1] - eigs[0]
    if disp:
        print(f'basin obj: {basin_obj}')

    cR = ula_signal.get_cov_matrix_toeplitz(csignal)
    theta_est1, _ = esprit.estimate_theta_toeplitz(cR)
    eigs = np.abs(esprit.eigs)[:2]
    true_obj = eigs[1] - eigs[0]
    if disp:
        print(f'true obj: {true_obj}')

    num_queries = np.sum(np.array(ula_signal.depths)*np.array(ula_signal.n_samples)) + ula_signal.n_samples[0]
    # Compute the maximum single query
    max_single_query = np.max(ula_signal.depths)

    ret_dict = {'theta_est': theta_est, 'theta_est1': theta_est1, 
                'error': np.abs(np.sin(theta)-np.sin(theta_est)), 
                'error1': np.abs(np.sin(theta)-np.sin(theta_est1)), 
                'queries': num_queries, 'depth': max_single_query,
                'true_obj': true_obj, 'basin_obj': basin_obj,
                'x_star': x_star, 'avg_nfev': avg_nfev/len(signs_to_try)}
    
    return ret_dict

csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=True, optimize=True, disp=True)

number of signs to try: 105
[ 1.   -1.    1.   -1.   -0.75  1.   -1.    1.   -1.    1.  ]
number fev: 20
[ 1.  1. -1.  1.  1.  1.  1. -1. -1. -1.]
number fev: 20
[ 1. -1. -1. -1.  1.  1.  1. -1. -1.  1.]
number fev: 20
[ 1.          0.90836316  0.55365233  0.9753466  -0.81321566 -1.
  1.          0.91705262  1.         -1.        ]
number fev: 20
[ 1. -1.  1.  1.  1.  1. -1. -1. -1. -1.]
number fev: 20
[ 1.         -0.17469775 -1.          1.          1.          1.
  1.         -1.         -1.         -1.        ]
number fev: 20
[ 1.         -1.         -1.         -1.          1.          1.
  1.         -0.91826778 -1.         -1.        ]
number fev: 20
[ 1.          1.          0.77581275 -1.          0.41862661 -1.
  0.47812342  0.5022376   0.10813438 -0.8371915 ]
number fev: 20
[ 1.         1.        -1.         1.         1.         0.8652079
  1.        -1.        -0.9256271 -1.       ]
number fev: 20
[ 1.          1.         -1.          1.          0.44608908 -1.
 -0.9456516

{'theta_est': 0.41067847866826507,
 'theta_est1': 0.4106816576390792,
 'error': 0.00040061197327528886,
 'error1': 0.0003976973353808644,
 'queries': 3069,
 'depth': 256,
 'true_obj': -588.2869394067335,
 'basin_obj': -602.20771133023,
 'x_star': array([ 1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1.]),
 'avg_nfev': 20.0}

In [441]:
np.random.seed(42)

avals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
avals = [np.random.uniform(0.1, 0.9)]
# avals = [0.5, 0.6, 0.7, 0.8, 0.9]
narray = [2]*10
heavy_signs = get_heavy_signs(narray, len(narray)**2)
num_queries = np.zeros(len(avals), dtype=int)
max_single_query = np.zeros(len(avals), dtype=int)

num_mc=100
thetas = np.zeros((len(avals), num_mc))
errors = np.zeros((len(avals), num_mc))
thetas1 = np.zeros((len(avals), num_mc))
errors1 = np.zeros((len(avals), num_mc))

basin_obj = np.zeros((len(avals), num_mc))
true_obj = np.zeros((len(avals), num_mc))

ula_signal = TwoqULASignal(M=narray, C=3)
esprit = ESPIRIT()

for j,a in enumerate(avals):
    theta = np.arcsin(a)
    print(f'theta = {theta}')
    for i in tqdm(range(num_mc)):
        res = csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=True, optimize=False, disp=False)
        thetas[j][i] = res['theta_est']
        errors[j][i] = res['error']

        thetas1[j][i] = res['theta_est1']
        errors1[j][i] = res['error1']

        basin_obj[j][i] = res['basin_obj']
        true_obj[j][i] = res['true_obj']
    num_queries[j] = res['queries']
    max_single_query[j] = res['depth']
    avg_nfev = res['avg_nfev']
    
    print(f'average nfev: {avg_nfev}')

    print(f'constant factors query and depth: {np.percentile(errors[j], 95) * num_queries[j]}, {np.percentile(errors[j], 95) * max_single_query[j]}')




theta = 0.41111546403370536


100%|██████████| 100/100 [11:14<00:00,  6.74s/it]

average nfev: 0.0
constant factors query and depth: 1145.7349737226452, 95.5245573271445


In [442]:
np.array(basin_obj[0])/np.array(true_obj[0]), np.sum(np.where(np.array(basin_obj[0])/np.array(true_obj[0]) >= 0.99, 1, 0))/len(basin_obj[0])

(array([0.97394572, 0.78857808, 0.70984506, 1.35943527, 0.84115242,
        0.97229596, 0.80033804, 0.70494123, 0.71259293, 0.67072013,
        0.94714057, 0.97650247, 0.65923025, 0.88176917, 3.70400716,
        0.94340493, 0.64933107, 1.12275523, 0.84932203, 0.76927282,
        0.94792228, 0.67498163, 0.74712531, 0.91828975, 0.84339047,
        0.74995258, 0.67025054, 0.98328603, 1.02550415, 0.76544046,
        0.77149959, 1.16078458, 1.1839647 , 1.00054327, 0.9334595 ,
        1.05176994, 0.68275347, 0.99251594, 1.48045577, 0.60557223,
        0.90177111, 0.75637042, 1.39410011, 0.95576077, 0.65371491,
        0.93138948, 0.77843755, 1.00635007, 1.03111657, 1.05859077,
        0.69022002, 0.67536517, 0.74429888, 0.85902698, 0.92630612,
        0.84331777, 0.97868118, 0.8204091 , 0.82693401, 1.50654798,
        0.73063552, 0.65530273, 0.71953501, 0.83381323, 1.02143068,
        0.79645327, 0.79870137, 0.86288832, 0.79058144, 0.63889126,
        0.72960453, 1.2009922 , 0.84233911, 0.81

In [443]:
np.random.seed(42)

avals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
avals = [np.random.uniform(0.1, 0.9)]
# avals = [0.5, 0.6, 0.7, 0.8, 0.9]
narray = [2]*10
heavy_signs = get_heavy_signs(narray, len(narray)**2)
num_queries = np.zeros(len(avals), dtype=int)
max_single_query = np.zeros(len(avals), dtype=int)

num_mc=100
thetas = np.zeros((len(avals), num_mc))
errors = np.zeros((len(avals), num_mc))
thetas1 = np.zeros((len(avals), num_mc))
errors1 = np.zeros((len(avals), num_mc))

basin_obj = np.zeros((len(avals), num_mc))
true_obj = np.zeros((len(avals), num_mc))

ula_signal = TwoqULASignal(M=narray, C=3)
esprit = ESPIRIT()

for j,a in enumerate(avals):
    theta = np.arcsin(a)
    print(f'theta = {theta}')
    for i in tqdm(range(num_mc)):
        res = csae_with_local_minimization(theta, ula_signal, esprit, heavy_signs, sample=True, optimize=True, method='L-BFGS-B', maxiter=2, disp=False)
        thetas[j][i] = res['theta_est']
        errors[j][i] = res['error']

        thetas1[j][i] = res['theta_est1']
        errors1[j][i] = res['error1']

        basin_obj[j][i] = res['basin_obj']
        true_obj[j][i] = res['true_obj']
    num_queries[j] = res['queries']
    max_single_query[j] = res['depth']
    avg_nfev = res['avg_nfev']
    
    print(f'average nfev: {avg_nfev}')

    print(f'constant factors query and depth: {np.percentile(errors[j], 95) * num_queries[j]}, {np.percentile(errors[j], 95) * max_single_query[j]}')

theta = 0.41111546403370536


  3%|▎         | 3/100 [18:32<10:12:34, 378.91s/it]

In [ ]:
np.array(basin_obj[0])/np.array(true_obj[0]), np.sum(np.where(np.array(basin_obj[0])/np.array(true_obj[0]) >= 0.99, 1, 0))/len(basin_obj[0])

(array([1.        , 1.04908198, 1.07908478, 1.01878539, 1.        ,
        1.        , 1.        , 1.00587482, 1.05254145, 1.        ,
        1.0269506 , 1.00283814, 1.00000304, 1.00521395, 1.        ,
        1.21275735, 1.05205596, 1.        , 1.03487204, 1.00389896,
        1.06483181, 1.42074136, 1.        , 1.        , 1.09763843,
        1.00013881, 1.14519365, 1.        , 1.02335159, 1.02727629,
        1.34199423, 1.00732311, 1.00030042, 1.        , 1.11425911,
        1.        , 1.17481194, 1.        , 1.        , 1.2487136 ,
        1.        , 1.        , 1.00858749, 1.01784644, 1.02740757,
        1.01378431, 1.00401606, 1.0445105 , 1.        , 1.08524829,
        1.01154633, 1.45948435, 2.12022093, 1.07651501, 1.0649688 ,
        1.        , 3.35451656, 1.01605685, 1.07219296, 1.16846807,
        1.07731426, 1.        , 1.0763753 , 1.01082878, 1.05849682,
        1.00309792, 1.03935751, 1.0757126 , 1.01300493, 1.        ,
        1.        , 1.00382388, 1.08902588, 1.01

In [434]:
Q = 7
lengths = []
for q in range(2, Q + 1):
    narray = [2]*(2*q)
    heavy_signs = get_heavy_signs(narray, len(narray)**2)
    signs_to_try = []
    for a in heavy_signs.keys():
        signs = heavy_signs[a].keys()
        signs_to_try += signs
    signs_to_try = list(set(signs_to_try))
    lengths.append(len(signs_to_try))

lengths

[14, 47, 108, 222, 434, 697]